In [2]:
!pip install rank-bm25
!pip install sentence-transformers

In [8]:
import numpy as np
from rank_bm25 import BM25Okapi
from sentence_transformers import SentenceTransformer, CrossEncoder, util
import torch

# 1. 가상의 지식 베이스 (Knowledge Base)
documents = [
    "DeBERTa는 상대적 위치 정보를 활용하여 문장의 미묘한 차이를 구분합니다.",
    "올림픽은 4년마다 열리는 국제 종합 스포츠 대회입니다.",
    "광합성은 식물이 빛 에너지를 이용해 포도당을 만드는 과정입니다.",
    "BPE 토크나이저는 빈도 기반으로 단어를 조각내어 사전을 만듭니다.",
    "트랜스포머 모델은 어텐션 메커니즘을 기반으로 작동합니다.",
    "GPU는 병렬 연산에 특화된 하드웨어입니다.",
    "자연어 처리는 컴퓨터가 인간 언어를 이해하도록 하는 분야입니다.",
    "검색 엔진은 대규모 문서 집합에서 정보를 찾는 시스템입니다.",
    "언어 모델은 다음 단어를 예측하는 방식으로 학습됩니다.",
    "제주도는 화산 활동으로 형성된 섬이며 한라산이 가장 높은 산입니다.",
    "비트코인은 블록체인 기술을 기반으로 한 탈중앙화 디지털 자산입니다.",
    "리랭커는 검색된 문서들 중 가장 적절한 답변을 상단으로 재정렬합니다.",
    "김치는 발효 과정을 통해 유산균이 생성되는 한국의 전통 음식입니다.",
    "태양은 수소와 헬륨으로 이루어진 항성으로 지구에 빛과 열을 제공합니다.",
    "vLLM은 PagedAttention을 통해 GPU 메모리 효율을 극대화합니다.",
    "TF-IDF는 단어의 빈도를 측정하지만 문맥적 의미를 파악하지 못합니다.",
    "RAG는 외부 지식을 가져와 LLM의 할루시네이션을 방지하는 기술입니다.",
    "피카소는 입체파를 창시한 스페인 출신의 화가입니다.",
    "양자역학은 미시 세계의 물리 현상을 설명하는 이론입니다.",
    "아마존 열대우림은 세계에서 가장 큰 열대우림 지역입니다.",
    "베토벤은 독일 출신의 작곡가로 교향곡 9번이 유명합니다.",
]

# 2. Retrieval 단계 - (1) Sparse Retrieval (BM25)
tokenized_corpus = [doc.split(" ") for doc in documents]
bm25 = BM25Okapi(tokenized_corpus)

# 2. Retrieval 단계 - (2) Dense Retrieval (Semantic Search)
# 프로젝트에서 사용하신 MiniLM과 같은 계열의 모델을 사용합니다.
bi_encoder = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
doc_embeddings = bi_encoder.encode(documents, convert_to_tensor=True)

# 3. Re-ranking 단계 - Cross-Encoder (DeBERTa v3 style)
# 정교한 비교를 위해 Cross-Encoder 모델을 로드합니다.
reranker = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

def hybrid_search(query, top_k=3):
    print(f"\n[질문]: {query}")

    # (A) BM25 검색
    tokenized_query = query.split(" ")
    bm25_scores = bm25.get_scores(tokenized_query)

    # (B) Semantic 검색
    query_embedding = bi_encoder.encode(query, convert_to_tensor=True)
    hits = util.semantic_search(query_embedding, doc_embeddings, top_k=len(documents))[0]
    semantic_scores = {hit['corpus_id']: hit['score'] for hit in hits}

    # (C) Hybrid 정렬 (단순 가중합 예시)
    hybrid_results = []
    for i in range(len(documents)):
        # 점수 정규화 (실무에선 RRF 방식을 주로 사용)
        combined_score = (bm25_scores[i] * 0.5) + (semantic_scores[i] * 0.5)
        hybrid_results.append((i, combined_score))

    hybrid_results = sorted(hybrid_results, key=lambda x: x[1], reverse=True)[:top_k]
    retrieved_docs = [documents[i] for i, score in hybrid_results]

    print(f"--- 1차 검색 완료 (Top {top_k}) ---")
    for doc in retrieved_docs:
        print(f"-> {doc}")

    # 4. Re-ranking (DeBERTa 방식의 정밀 검사)
    # 질문과 검색된 각 문서를 쌍(Pair)으로 묶어 점수를 매깁니다.
    sentence_pairs = [[query, doc] for doc in retrieved_docs]
    rerank_scores = reranker.predict(sentence_pairs)

    # 결과 정렬
    reranked_results = sorted(zip(retrieved_docs, rerank_scores), key=lambda x: x[1], reverse=True)

    print(f"\n--- 2차 리랭킹 완료 (최종 결과) ---")
    for doc, score in reranked_results:
        print(f"[{score:.4f}] {doc}")

    print("\n================================================================\n")

# 실습 실행
# hybrid_search("BPE와 DeBERTa의 차이점이 뭐야?")
# hybrid_search("GPU 메모리 관리 기법에 대해 알려줘")

# 의미 기반 검색(Dense)이 제대로 작동하는지 / RAG 문서가 1위로 오는지 / 리랭커가 “할루시네이션 방지” 문서를 최상단에 두는지
# => 의미 검색 성능 테스트용 질문
hybrid_search("RAG는 왜 필요한가요?")

# BM25가 정확히 매칭되는지 / 리랭킹 단계가 점수를 더 확실히 올리는지
# => 파이프라인 전체가 정상 작동하는지 확인하는 기본 테스트
hybrid_search("리랭커는 무엇을 개선하나요?")

# 질문 표현이 문서와 완전히 동일하지 않음 / 의미 기반 검색이 필요한 상황
# => BM25 vs Dense 차이를 보기 좋은 질문
hybrid_search("BPE 토크나이저는 어떻게 동작하나요?")

# GPU 관련 문서와 헷갈리지 않는지 / Hybrid 점수가 제대로 분리되는지
# => 헷갈리는 문서 사이에서 정밀도 확인 테스트
hybrid_search("vLLM의 메모리 최적화 방식은?")

# "어텐션 메커니즘" 문서가 정확히 올라오는지 / 자연어 표현 차이를 얼마나 잘 이해하는지
# 키워드 일부 일치 + 의미 유사 필요
hybrid_search("트랜스포머 모델의 핵심 메커니즘은?")


Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Loading weights:   0%|          | 0/105 [00:00<?, ?it/s]

BertForSequenceClassification LOAD REPORT from: cross-encoder/ms-marco-MiniLM-L-6-v2
Key                          | Status     |  | 
-----------------------------+------------+--+-
bert.embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.



[질문]: RAG는 왜 필요한가요?
--- 1차 검색 완료 (Top 3) ---
-> RAG는 외부 지식을 가져와 LLM의 할루시네이션을 방지하는 기술입니다.
-> 비트코인은 블록체인 기술을 기반으로 한 탈중앙화 디지털 자산입니다.
-> 트랜스포머 모델은 어텐션 메커니즘을 기반으로 작동합니다.

--- 2차 리랭킹 완료 (최종 결과) ---
[8.7466] RAG는 외부 지식을 가져와 LLM의 할루시네이션을 방지하는 기술입니다.
[8.5698] 트랜스포머 모델은 어텐션 메커니즘을 기반으로 작동합니다.
[8.2892] 비트코인은 블록체인 기술을 기반으로 한 탈중앙화 디지털 자산입니다.



[질문]: 리랭커는 무엇을 개선하나요?
--- 1차 검색 완료 (Top 3) ---
-> 리랭커는 검색된 문서들 중 가장 적절한 답변을 상단으로 재정렬합니다.
-> 트랜스포머 모델은 어텐션 메커니즘을 기반으로 작동합니다.
-> RAG는 외부 지식을 가져와 LLM의 할루시네이션을 방지하는 기술입니다.

--- 2차 리랭킹 완료 (최종 결과) ---
[8.1499] 트랜스포머 모델은 어텐션 메커니즘을 기반으로 작동합니다.
[8.1295] 리랭커는 검색된 문서들 중 가장 적절한 답변을 상단으로 재정렬합니다.
[7.8659] RAG는 외부 지식을 가져와 LLM의 할루시네이션을 방지하는 기술입니다.



[질문]: BPE 토크나이저는 어떻게 동작하나요?
--- 1차 검색 완료 (Top 3) ---
-> BPE 토크나이저는 빈도 기반으로 단어를 조각내어 사전을 만듭니다.
-> GPU는 병렬 연산에 특화된 하드웨어입니다.
-> vLLM은 PagedAttention을 통해 GPU 메모리 효율을 극대화합니다.

--- 2차 리랭킹 완료 (최종 결과) ---
[8.1442] BPE 토크나이저는 빈도 기반으로 단어를 조각내어 사전을 만듭니다.
[5.9059] vLLM은 PagedAttention을 통해 GPU 메모리 효율을 극대화합니다.
[3.3807] GPU는 병렬 연산에 특화된 하드웨어입니